### Initial Imports:

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
pd.set_option("display.max_colwidth", 300)

import requests
from dotenv import load_dotenv
from datetime import datetime,timedelta
import math

import json
from web3.auto import w3

### Additional Installation:

In [2]:
# !pip install yfinance
import yfinance as yf

### Download Stock Data from Yahoo Finance:

In [3]:
# Get Recent Business Date:
from datetime import datetime,timedelta
from pandas.tseries.offsets import BDay
recent_bus_day = (datetime.today()-BDay(2)).strftime("%Y-%m-%d")

# Auto-Update Time Interval
start=(datetime.today()-timedelta(days=3)).strftime("%Y-%m-%d")
end=datetime.today().strftime("%Y-%m-%d")
print(f"start date {start}")
print(f"end date {end}")
print(f"last business day {recent_bus_day}")

start date 2021-04-28
end date 2021-05-01
last business day 2021-04-29


In [4]:
# Dowload BRK Data from Yahoo
ticker = "BRK-A"
brk_yfinance = yf.download(ticker, start, end)
# Store in Dataframe
BRK=brk_yfinance["Adj Close"].to_frame(name="BRK")
print(BRK)

[*********************100%***********************]  1 of 1 completed
                 BRK
Date                
2021-04-27  411400.0
2021-04-28  410988.0
2021-04-29  417905.0
2021-04-30  412500.0


In [5]:
# Dowload ETH Data from Yahoo
ticker = "ETH-USD"
eth_yfinance = yf.download(ticker, start, end)
# Store in Dataframe
ETH=eth_yfinance["Adj Close"].to_frame(name="ETH")
print(ETH)

[*********************100%***********************]  1 of 1 completed
                    ETH
Date                   
2021-04-27  2662.865234
2021-04-28  2746.380127
2021-04-29  2756.876953


### Calculate BRK to ETH Ratio for the Last Business Day:

In [6]:
# Concat DataFrames
stock_data=pd.concat([BRK,ETH],axis='columns',join="inner").reset_index()
# Calculate and Extract Recent Ratio (the recent business day)
stock_data["ratio"]=stock_data["BRK"]/stock_data["ETH"]
ratio_df = stock_data.loc[stock_data["Date"]==recent_bus_day]
print(ratio_df)

        Date       BRK          ETH       ratio
2 2021-04-29  417905.0  2756.876953  151.586381


In [7]:
# Store Recent BRK to ETH Ratio (the recent business day)
# Always Round Up!!
import math
num_ETH = math.ceil(ratio_df["ratio"])
share_price = int(ratio_df['BRK'].to_numpy())
ETH_price = int(ratio_df['ETH'].to_numpy())

print(f"BRK price of last business date: {share_price}")
print(f"ETH price of last business date: {ETH_price}")   
print(f"Required number of ETH: {num_ETH}")

BRK price of last business date: 417905
ETH price of last business date: 2756
Required number of ETH: 152


### Initialize & Deploy Solidity Contract: GoalDeployer

In [74]:
def initContract():
    with open(Path("GoalDeployer.json")) as json_file:
        abi = json.load(json_file)
    return w3.eth.contract(address=w3.toChecksumAddress("0x60604f12E6c963B0dd7B30f474f2d3FC6ce16Eb6"), abi=abi)        

In [75]:
GoalDeployer = initContract()

### Sending Updated Data to Solidity Contract: GoalDeployer

In [77]:
def update_goal(date, share_price, ETH_price, ratio):
    tx_hash = GoalDeployer.functions.Update(date, share_price, ETH_price, ratio).transact(
        {"from": w3.eth.accounts[0]}
    )
    receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    print(f"Event log is updated for {date}")
    return receipt

In [78]:
update_goal(recent_bus_day, share_price, ETH_price, num_ETH)

Event log is updated for 2021-04-29


AttributeDict({'transactionHash': HexBytes('0x68c56157775827023f2292df42a1446f8202ce9f28b434c9c5b9fc33069d7dc1'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0xc528c67f0ff643d6c7665f5542204849a1ca2b0a69214317dc576a3af0e87cda'),
 'blockNumber': 482,
 'from': '0xA850442918Cc83e9Db3654DC353717a802DF2b85',
 'to': '0x60604f12E6c963B0dd7B30f474f2d3FC6ce16Eb6',
 'gasUsed': 108441,
 'cumulativeGasUsed': 108441,
 'contractAddress': None,
 'logs': [AttributeDict({'logIndex': 0,
   'transactionIndex': 0,
   'transactionHash': HexBytes('0x68c56157775827023f2292df42a1446f8202ce9f28b434c9c5b9fc33069d7dc1'),
   'blockHash': HexBytes('0xc528c67f0ff643d6c7665f5542204849a1ca2b0a69214317dc576a3af0e87cda'),
   'blockNumber': 482,
   'address': '0x60604f12E6c963B0dd7B30f474f2d3FC6ce16Eb6',
   'data': '0x000000000000000000000000000000000000000000000000000000000000008000000000000000000000000000000000000000000000000000000000000660710000000000000000000000000000000000000000000000000000000000000ac40000000000

In [26]:
def get_goal(date):
    price_filter = GoalDeployer.events.goal.createFilter(
        fromBlock="0x0", argument_filters={"date": date}
    )
    return price_filter.get_all_entries()

In [28]:
get_goal("2021-04-29")

[]

### Initialize Solidity Contract: BHCoinSale

In [14]:
def initContract():
    with open(Path("BHCoinSale.json")) as json_file:
        abi_2 = json.load(json_file)
    return w3.eth.contract(address="0x688bE037cd639254C082Ca1e1bB88F9817174183", abi=abi_2) 

In [15]:
BHCoinSale = initContract()

### Get Updated Sale Status from Solidity Contract: GoalDeployer

In [31]:
get_finalized_status = BHCoinSale.functions.finalized().call()
get_finalized_status

False

In [34]:
weiRasied_status = BHCoinSale.functions.weiRaised().call()
weiRasied_status

0

### API to Broker

1. Firstly check the Crowd Sale Status,
2. If the Crowd Sale is **Finalized**, then check the `amount` of **weiRasied**.
3. Next, use the proceeds from the Crowd Sale to place trade with the broker.

### Testing: At the End of the Lock Period, Push NEW Price Data for User to Redeem:

In [45]:
# Assumed price data for testing purpose
end_BRK_price = 500000
end_ETH_price = 2000
end_num_ETH = math.ceil(end_BRK_price/end_ETH_price)
print(end_num_ETH)

250


In [46]:
update_goal('2022/5/1', end_BRK_price, end_ETH_price, end_num_ETH)

Event log is updated for 2022/5/1


AttributeDict({'transactionHash': HexBytes('0xba3e0caa87dc37d31e3aaa139e2c5de54119b7f66bbab521132ce4b67fe97c22'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0xd29974345df792d02a0277aa6ffd2bf8979955e6a826148501c1bdd5b64bbe67'),
 'blockNumber': 456,
 'from': '0xA850442918Cc83e9Db3654DC353717a802DF2b85',
 'to': '0x7d7FA8911824b2BDeD6E6C7693843Ffa9f938d23',
 'gasUsed': 93417,
 'cumulativeGasUsed': 93417,
 'contractAddress': None,
 'logs': [AttributeDict({'logIndex': 0,
   'transactionIndex': 0,
   'transactionHash': HexBytes('0xba3e0caa87dc37d31e3aaa139e2c5de54119b7f66bbab521132ce4b67fe97c22'),
   'blockHash': HexBytes('0xd29974345df792d02a0277aa6ffd2bf8979955e6a826148501c1bdd5b64bbe67'),
   'blockNumber': 456,
   'address': '0x7d7FA8911824b2BDeD6E6C7693843Ffa9f938d23',
   'data': '0x0000000000000000000000000000000000000000000000000000000000000080000000000000000000000000000000000000000000000000000000000007a12000000000000000000000000000000000000000000000000000000000000007d0000000000000